In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
import os
from sklearn import linear_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import Sequence

## Loading the Data

In [5]:
os.listdir("./input/")

['avito-demand-prediction', 'fasttext-russian-2m']

In [5]:
trainDF = pd.read_csv("./input/avito-demand-prediction/train.csv")
testDF = pd.read_csv("./input/avito-demand-prediction/test.csv")

trainDF.head()
#testDF.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


## Preprocessing

In [1]:
features = ["price","title","description"]
selectFrame = trainDF[ trainDF["deal_probability"] >= .9 ][features+["deal_probability"]]
selectFrame["price"] = selectFrame["price"].fillna(0)
selectFrame["title"] = selectFrame["title"].fillna("")
selectFrame["description"] = selectFrame["description"].fillna("")

zeroFrame = trainDF[trainDF["deal_probability"] <.9 ][features+["deal_probability"]]
zeroFrame["price"] = zeroFrame["price"].fillna(0)
zeroFrame["description"] = zeroFrame["description"].fillna("")
# Setting all del_prob to zero in this category. As we'd like to discriminate against it. 
zeroFrame["deal_probability"] = 0

# Combine the <.9 data and >= .9 data and shuffle the combined records. 
#
tmpFrame = trainDF
tmpFrame["price"] = tmpFrame["price"].fillna(0)
tmpFrame["description"] = tmpFrame["description"].fillna("")
combinedFrames = pd.concat([selectFrame,zeroFrame])

selectFrame
max_features = 20000
maxlen = 1000

descTokenizer = Tokenizer(num_words=max_features)
descTokenizer.fit_on_texts(combinedFrames["description"]) 

#descTokens =  pad_sequences(descTokenizer.texts_to_sequences(selectFrame["description"]),maxlen = maxlen )
#zeroTokens =  pad_sequences(descTokenizer.texts_to_sequences(zeroFrame["description"]),maxlen = maxlen )

NameError: name 'trainDF' is not defined

In [19]:
zeroFrame["price"].count()/selectFrame["price"].count()

16.22908302182842

In [15]:
combinedFrames["description"].shape[0]

1503424

In [21]:
class DescGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, nullClass, targetClass, descTokenizer, maxlen,batch_size=32,shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.nullClass = nullClass
        self.targetClass = targetClass
        self.descTokenizer = descTokenizer
        self.maxlen = maxlen
       
        self.shuffle = shuffle
        #self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.nullClass.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        nullBatch = self.nullClass.sample(n=self.batch_size)
        targetBatch = self.targetClass.sample(n=self.batch_size)
        combBatch = pd.concat( [nullBatch ,targetBatch] )
        
        #Todo suffle 
        
        combBatchTokens =  pad_sequences(self.descTokenizer.texts_to_sequences(combBatch["description"]),maxlen = self.maxlen )
        # Generate data
    

        return [combBatchTokens,combBatch["price"]], combBatch["deal_probability"]

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        #self.indexes = np.arange(len(self.list_IDs))
        #if self.shuffle == True:
            #np.random.shuffle(self.indexes)
            
dataGen = DescGenerator(zeroFrame,selectFrame,descTokenizer,maxlen)

## Modeling

In [24]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 300
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier
x = Embedding(max_features, embed_size)(inp)
x = LSTM(100, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)

priceInp = Input(shape=(1, ))
y = Dense(100,activation="sigmoid")(priceInp)
#catInp = Input(shape=(1, ))
x = Concatenate(axis=-1)([y,x])
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[inp,priceInp], outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1000, 300)    6000000     input_5[0][0]                    
__________________________________________________________________________________________________
lstm_layer (LSTM)               (None, 1000, 100)    160400      embedding_3[0][0]                
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
global_max

In [25]:
model.fit_generator(dataGen,steps_per_epoch=len(dataGen),epochs=10, verbose=1)

Epoch 1/10
  123/43958 [..............................] - ETA: 13:23:56 - loss: 0.6434 - acc: 0.4498

KeyboardInterrupt: 

In [14]:
deals = trainDF[trainDF["deal_probability"] >= .9 ][0:10]
dealTok = pad_sequences(descTokenizer.texts_to_sequences(deals["description"]),maxlen =maxlen )
np.set_printoptions(suppress=True)
res = model.predict([dealTok,deals["price"]])
print(res,deals["deal_probability"])

[[0.9986406 ]
 [0.97313595]
 [0.9775955 ]
 [0.9894316 ]
 [0.9769403 ]
 [0.9764564 ]
 [0.9842002 ]
 [0.9987431 ]
 [0.9904108 ]
 [0.98018503]] 65     1.0
66     1.0
207    1.0
254    1.0
332    1.0
451    1.0
666    1.0
704    1.0
807    1.0
814    1.0
Name: deal_probability, dtype: float64


In [2]:
deals = trainDF[trainDF["deal_probability"] < .9 ][0:10]
dealTok = pad_sequences(descTokenizer.texts_to_sequences(deals["description"]),maxlen =maxlen )
np.set_printoptions(suppress=True)
res = model.predict([dealTok,deals["price"]])
print(res,deals["deal_probability"])

NameError: name 'trainDF' is not defined

In [16]:
model.save("90_1th.h5")